In [ ]:
! pip install pandas unidecode
# TODO: organizer isso dps, e descrever o preocessamento
"""
primeiro cross matching
depois mathcing fuzzy/relaxado - matchings.txt
depois ajustes manuais
geramos dados_com_cdb
"""

# NAO RODAR CELULAS DESSE NOTEBOOK

Defaulting to user installation because normal site-packages is not writeable


'\nprimeiro cross matching\n'

In [ ]:
# cross.py
import pandas as pd

# Leitura dos dados
data = pd.read_csv("dados_filtrados.csv")
cdb = pd.read_csv("bares_cdb2025.csv")

# Normalização dos textos
data["NOME_FANTASIA"] = data["NOME_FANTASIA"].str.upper().str.strip()
data["ENDERECO_COMPLETO"] = data["ENDERECO_COMPLETO"].str.upper().str.strip()

cdb["Nome"] = cdb["Nome"].str.upper().str.strip()
cdb["Endereço"] = cdb["Endereço"].str.upper().str.strip()

# Conjuntos de referência
nomes_ref = set(data["NOME_FANTASIA"])
enderecos_ref = set(data["ENDERECO_COMPLETO"])

# Listas para armazenar resultados
nao_encontrados = []
encontrados = []

# Verificação de existência
for _, row in cdb.iterrows():
    nome = row["Nome"]
    endereco = row["Endereço"]

    if endereco in enderecos_ref:
        encontrados.append({
            "Nome": nome,
            "Endereço": endereco
        })
    else:
        nao_encontrados.append({
            "Nome": nome,
            "Endereço": endereco
        })

# Impressão dos não encontrados
for item in nao_encontrados:
    print(f'Não encontrado: {item["Nome"]} - {item["Endereço"]}')

# Salvando os resultados
pd.DataFrame(nao_encontrados).to_csv("nao_encontrados.csv", index=False, encoding="utf-8-sig")
pd.DataFrame(encontrados).to_csv("encontrados.csv", index=False, encoding="utf-8-sig")


In [ ]:
import pandas as pd
import re
from unidecode import unidecode

def normalizar(texto):
    if pd.isna(texto):
        return ""
    return unidecode(texto.strip().upper())

def extrair_rua(endereco):
    return normalizar(endereco.split(",")[0])

def nome_bate(nome_csv1, nome_csv2, nome_fantasia):
    palavras = [normalizar(p) for p in nome_csv1.split()]
    nome_csv2 = normalizar(nome_csv2)
    nome_fantasia = normalizar(nome_fantasia)
    return any(p in nome_csv2 or p in nome_fantasia for p in palavras)

def endereco_bate(end1, end2):
    return extrair_rua(end1) in normalizar(end2)

csv1 = pd.read_csv("nao_encontrados.csv") 
csv2 = pd.read_csv("dados_filtrados.csv")  

with open("matchings.txt", "w", encoding="utf-8") as output:
    for _, row1 in csv1.iterrows():
        nome1 = str(row1["Nome"])
        endereco1 = str(row1["Endereço"])

        output.write(f'{nome1},"{endereco1}"\n')
        output.write("matchings (pode haver mais de um):\n")

        achou = False
        for _, row2 in csv2.iterrows():
            nome_real = str(row2.get("NOME", ""))
            nome_fantasia = str(row2.get("NOME_FANTASIA", ""))
            endereco2 = str(row2.get("ENDERECO_COMPLETO", ""))

            if nome_bate(nome1, nome_real, nome_fantasia) and endereco_bate(endereco1, endereco2):
                output.write(f'"{endereco2}"{nome_fantasia, nome_real}\n')
                achou = True

        if not achou:
            output.write("Nenhum matching encontrado.\n")
        output.write("\n")


> Aqui fiz um ultimo matching mais flexivel para checar alguns bares - matching.txt

CANTINHO DA BAIANA,"AVENIDA ITAITE, 422, SÃO GERALDO, BELO HORIZONTE, MG, BRASIL"
matchings:
"AVENIDA ITAITE, 422, SAO GERALDO, BELO HORIZONTE, MG, BRASIL"('Estabelecimento sem nome', 'CANTINHO DA BAIANA COMERCIAL LTDA')

BAR DA LU,"RUA GERALDA MARINHO, 41, SÃO JOÃO BATISTA, BELO HORIZONTE, MG, BRASIL"
matchings:
"RUA GERALDA MARINHO, 41, SAO JOAO BATISTA, BELO HORIZONTE, MG, BRASIL"('BAR E RESTAURANTE DA LU', '38.029.984 LUZMARINA PEREIRA DAS NEVES')

TROPEIRO DO LISBOA,"AVENIDA LEONTINO FRANCISCO ALVES, 506, SERRA VERDE, BELO HORIZONTE, MG, BRASIL"
matchings:
"AVENIDA LEONTINO FRANCISCO ALVES, 810, SERRA VERDE, BELO HORIZONTE, MG, BRASIL"('REI DO TROPEIRO', 'REI DO TROPEIRO LTDA')
-> checado esse rei do tropeiro tem o mesmo cnpj

In [10]:
import pandas as pd

# Carregar os dados
data = pd.read_csv("dados_filtrados.csv")
cdb = pd.read_csv("bares_cdb2025.csv")

# Normalizar campos
data["NOME_FANTASIA"] = data["NOME_FANTASIA"].str.upper().str.strip()
data["ENDERECO_COMPLETO"] = data["ENDERECO_COMPLETO"].str.upper().str.strip()

cdb["Nome"] = cdb["Nome"].str.upper().str.strip()
cdb["Endereço"] = cdb["Endereço"].str.upper().str.strip()

# Criar sets para comparação
enderecos_ref = set(data["ENDERECO_COMPLETO"])

# Identificar bares encontrados
encontrados = []
for _, row in cdb.iterrows():
    nome = row["Nome"]
    endereco = row["Endereço"]
    if endereco in enderecos_ref:
        encontrados.append({"Nome": nome, "Endereço": endereco})

# Criar DataFrame dos encontrados com ID
df_encontrados = pd.DataFrame(encontrados).drop_duplicates(subset=["Endereço"]).reset_index(drop=True)
df_encontrados["CDB_ID"] = df_encontrados.index + 1
df_encontrados.to_csv("encontrados.csv", index=False, encoding="utf-8-sig")

# Criar dicionários para mapeamento
map_endereco_to_id = dict(zip(df_encontrados["Endereço"], df_encontrados["CDB_ID"]))
map_endereco_to_nome = dict(zip(df_encontrados["Endereço"], df_encontrados["Nome"]))

# Adicionar coluna CDB no dataframe original
data["CDB"] = data["ENDERECO_COMPLETO"].map(map_endereco_to_id).fillna(0).astype(int)

# Substituir NOME_FANTASIA pelo nome do CDB, se encontrado
data["NOME_FANTASIA"] = data.apply(
    lambda row: map_endereco_to_nome.get(row["ENDERECO_COMPLETO"], row["NOME_FANTASIA"]),
    axis=1
)

# Salvar CSV final
data.to_csv("dados_com_cdb.csv", index=False, encoding="utf-8-sig")


In [11]:
df_encontrados = pd.DataFrame(encontrados).drop_duplicates(subset=["Endereço"]).reset_index(drop=True)
df_encontrados["CDB_ID"] = df_encontrados.index + 1

# Reorganizar colunas: CDB_ID primeiro
cols = ["CDB_ID"] + [col for col in df_encontrados.columns if col != "CDB_ID"]
df_encontrados = df_encontrados[cols]

# Salvar CSV com CDB_ID como primeira coluna
df_encontrados.to_csv("encontrados.csv", index=False, encoding="utf-8-sig")

In [1]:
import pandas as pd

# Lê o CSV original com os dados existentes
dados_existentes = pd.read_csv("dados_com_cdb.csv")

# Lê o CSV dos bares não encontrados, contendo CDB_ID, Nome e Endereço
novos_bares = pd.read_csv("cdb_notfound.csv")

# Renomeia as colunas para se adequar ao padrão do CSV final
novos_bares.rename(columns={
    "CDB_ID": "CDB",
    "Nome": "NOME",
    "Endereço": "ENDERECO_COMPLETO"
}, inplace=True)

# Adiciona as colunas faltantes com os valores definidos
novos_bares["ID_ATIV_ECON_ESTABELECIMENTO"] = "indisponivel"
novos_bares["CNAE_PRINCIPAL"] = "indisponivel"
novos_bares["DATA_INICIO_ATIVIDADE"] = "indisponivel"
novos_bares["IND_POSSUI_ALVARA"] = "SIM"
novos_bares["NOME_FANTASIA"] = "indisponivel"
novos_bares["GEOMETRIA"] = "indisponivel"

# Reorganiza as colunas na mesma ordem do arquivo original
colunas_ordenadas = [
    "ID_ATIV_ECON_ESTABELECIMENTO",
    "CNAE_PRINCIPAL",
    "DATA_INICIO_ATIVIDADE",
    "IND_POSSUI_ALVARA",
    "ENDERECO_COMPLETO",
    "NOME",
    "NOME_FANTASIA",
    "GEOMETRIA",
    "CDB"
]
novos_bares = novos_bares[colunas_ordenadas]

# Concatena os dados
dados_atualizados = pd.concat([dados_existentes, novos_bares], ignore_index=True)

# Salva no mesmo arquivo
dados_atualizados.to_csv("dados_com_cdb.csv", index=False, encoding="utf-8-sig")


In [4]:
import pandas as pd
# padronizar data types

df = pd.read_csv(
    "dados_com_cdb.csv",
    dtype={
        "ID_ATIV_ECON_ESTABELECIMENTO": str,
        "CNAE_PRINCIPAL": str,
        "IND_POSSUI_ALVARA": str,
        "ENDERECO_COMPLETO": str,
        "NOME": str,
        "NOME_FANTASIA": str,
        "GEOMETRIA": str,
        "CDB": "Int64",  
    },
    parse_dates=["DATA_INICIO_ATIVIDADE"],  # converte para datetime64
    dayfirst=True  
)
df["DATA_INICIO_ATIVIDADE"] = pd.to_datetime(df["DATA_INICIO_ATIVIDADE"], dayfirst=True, errors="coerce")


print(df.dtypes)

ID_ATIV_ECON_ESTABELECIMENTO            object
CNAE_PRINCIPAL                          object
DATA_INICIO_ATIVIDADE           datetime64[ns]
IND_POSSUI_ALVARA                       object
ENDERECO_COMPLETO                       object
NOME                                    object
NOME_FANTASIA                           object
GEOMETRIA                               object
CDB                                      Int64
dtype: object
